# 01 - EDA YouTube Toxic Comments

En este notebook realizamos el **Análisis Exploratorio de Datos (EDA)** del dataset de comentarios de YouTube etiquetados por toxicidad.

## Objetivos

- Entender la estructura del dataset (filas, columnas, tipos de datos).
- Analizar la distribución de las diferentes etiquetas de toxicidad (multietiqueta).
- Estudiar la longitud y características básicas de los comentarios.
- Detectar posibles problemas: valores nulos, duplicados, desbalanceo de clases.
- Analizar la relación entre tipos de toxicidad.
- Explorar diferencias entre comentarios tóxicos y no tóxicos.
- Obtener insights que nos ayuden a tomar decisiones de preprocesamiento y modelado.


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
import re

plt.style.use("ggplot")
sns.set(font_scale=1.1)

pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 100)


## 1. Carga de datos

En esta sección:

- Cargamos el CSV que contiene los comentarios de YouTube.
- Comprobamos el número de filas y columnas.
- Visualizamos una muestra de las primeras filas para tener una idea rápida del contenido.


In [3]:
DATA_PATH = "../../data/youtoxic_english_1000.csv"

df = pd.read_csv(DATA_PATH)

print("Tamaño del DataFrame (filas, columnas):", df.shape)
df.head()


Tamaño del DataFrame (filas, columnas): (1000, 15)


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,"If only people would just take a step back and not make this case about them, because it wasn't about anyone except the two people in that situation. To lump yourself into this mess and take matt...",False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to apprehend. They are trained to shoot to kill. And I thank Wilson for killing that punk bitch.,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' banners being held by white cunts is kinda patronizing and ironically racist. could they have not come up with somethin better.. or is it just what w...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do not like police officers. They are called Criminals and its the reason we have police officers. The fact that Criminals do not like police officers i...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should have not been shot 6 extra time. Shoot him once if hes attacking you and that would stop his attack. Shoot him twice if he's still attacking you, but s...",False,False,False,False,False,False,False,False,False,False,False,False


## 2. Información general del dataset

En esta sección analizamos:

- Tipos de datos de cada columna (`df.info()`).
- Número de valores nulos por columna.
- Número de valores únicos por columna.

Esto nos ayuda a entender la calidad de los datos y posibles problemas iniciales.


In [4]:
print(">>> Info del DataFrame")
print("-" * 80)
df.info()

print("\n>>> Valores nulos por columna")
print("-" * 80)
display(df.isnull().sum())

print("\n>>> Valores únicos por columna")
print("-" * 80)
display(df.nunique())


>>> Info del DataFrame
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   bool  
 4   IsAbusive        1000 non-null   bool  
 5   IsThreat         1000 non-null   bool  
 6   IsProvocative    1000 non-null   bool  
 7   IsObscene        1000 non-null   bool  
 8   IsHatespeech     1000 non-null   bool  
 9   IsRacist         1000 non-null   bool  
 10  IsNationalist    1000 non-null   bool  
 11  IsSexist         1000 non-null   bool  
 12  IsHomophobic     1000 non-null   bool  
 13  IsReligiousHate  1000 non-null   bool  
 14  IsRadicalism     1000 non-null   bool  
dtypes: bool(12), object(

CommentId          0
VideoId            0
Text               0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsSexist           0
IsHomophobic       0
IsReligiousHate    0
IsRadicalism       0
dtype: int64


>>> Valores únicos por columna
--------------------------------------------------------------------------------


CommentId          1000
VideoId              13
Text                997
IsToxic               2
IsAbusive             2
IsThreat              2
IsProvocative         2
IsObscene             2
IsHatespeech          2
IsRacist              2
IsNationalist         2
IsSexist              2
IsHomophobic          1
IsReligiousHate       2
IsRadicalism          1
dtype: int64

## 3. Identificación de tipos de columnas

Clasificamos las columnas en:

- **Identificadores**: columnas que identifican el comentario y el vídeo.
- **Texto**: columna que contiene el comentario en sí.
- **Etiquetas**: columnas binarias que indican diferentes tipos de toxicidad (`Is...`).

Esta separación es útil para el análisis y para el posterior preprocesamiento.


In [5]:
all_columns = df.columns.tolist()

id_cols = ["CommentId", "VideoId"]
text_col = "Text"
label_cols = [c for c in df.columns if c.startswith("Is")]

print("Todas las columnas:", all_columns)
print("\nColumnas ID:", id_cols)
print("Columna de texto:", text_col)
print("Columnas de etiquetas:", label_cols)


Todas las columnas: ['CommentId', 'VideoId', 'Text', 'IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']

Columnas ID: ['CommentId', 'VideoId']
Columna de texto: Text
Columnas de etiquetas: ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']


## 4. Análisis de duplicados

En esta sección buscamos:

- Comentarios con **`CommentId` duplicado**.
- Comentarios con **texto (`Text`) duplicado**.

Los duplicados pueden introducir sesgos en el entrenamiento del modelo, por eso es importante detectarlos desde el EDA.


In [6]:
# Duplicados por CommentId
dup_comment_id = df.duplicated(subset=["CommentId"]).sum()

# Duplicados por texto exacto
dup_text = df.duplicated(subset=["Text"]).sum()

print(f"Número de comentarios con CommentId duplicado: {dup_comment_id}")
print(f"Número de comentarios con texto duplicado: {dup_text}")

if dup_comment_id > 0:
    print("\n>>> Ejemplos de CommentId duplicados")
    display(df[df.duplicated(subset=["CommentId"], keep=False)].head())

if dup_text > 0:
    print("\n>>> Ejemplos de textos duplicados")
    display(df[df.duplicated(subset=["Text"], keep=False)].head())


Número de comentarios con CommentId duplicado: 0
Número de comentarios con texto duplicado: 3

>>> Ejemplos de textos duplicados


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
592,UgiXm5jxvkdIxHgCoAEC,cT14IbTDW2c,RUN THEM OVER,True,True,False,True,False,False,False,False,False,False,False,False
642,Ugxen2QgJYhNiRrMegR4AaABAg,cT14IbTDW2c,run them over,True,True,True,False,False,False,False,False,False,False,False,False
657,UgxXtUmfp0rdwXB8qld4AaABAg,cT14IbTDW2c,run them over,True,True,False,True,False,False,False,False,False,False,False,False
677,UgyjhPsMlWKlFNmG-h94AaABAg,cT14IbTDW2c,run them over,True,True,False,True,False,False,False,False,False,False,False,False
699,UgzFZGnqcjZcW7wejI54AaABAg,cT14IbTDW2c,RUN THEM OVER,True,True,False,True,False,False,False,False,False,False,False,False
